# Trajectory Optimization of a monopod
## Generate training data for the ML neural net

### The general idea...

Using trajectory optimization, locally optimal solutions can be found for a monopod performing a task. Using these trajectories, a ML model can be trained to predict, for a given state, what the control action (torque) should be. 

## Derive Equations of Motion



In [2]:
%reset
# DERIVE THE EOMs SYMBOLICALLY ----------------------------------------------------------------------------------------------

# import libraries
import sympy as sym
import numpy as np

sym.init_printing()
from IPython.display import display #for pretty printing

# create symbolic variables

# system parameters
g = sym.symbols('g')
mb,ml1,ml2 = sym.symbols(['m_{body}','m_{leg1}','m_{leg2}']) # mass
lb,ll1,ll2 = sym.symbols(['l_{body}','l_{leg1}','l_{leg2}']) # length
Inb,Inl1,Inl2 = sym.symbols(['I_{body}','I_{leg1}','I_{leg2}']) # moment of intertia

# generalized coordinates
x,y,thb,thlt,thlb = sym.symbols(['x','y','\\theta_{body}','\\theta_{legtop}','\\theta_{legbot}']) 
dx,dy,dthb,dthlt,dthlb = sym.symbols(['\dot{x}','\dot{y}','\dot{\\theta}_{body}','\dot{\\theta}_{legtop}','\dot{\\theta}_{legbot}']) 
ddx,ddy,ddthb,ddthlt,ddthlb = sym.symbols(['\ddot{x}','\ddot{y}','\ddot{\\theta}_{body}','\ddot{\\theta}_{legtop}','\ddot{\\theta}_{legbot}']) 

q = sym.Matrix([[x],[y],[thb],[thlt],[thlb]])
dq = sym.Matrix([[dx],[dy],[dthb],[dthlt],[dthlb]])
ddq = sym.Matrix([[ddx],[ddy],[ddthb],[ddthlt],[ddthlb]])

# forces
# total joint action = actuator + rebound, but that will be dealt with elsewhere
tauh,tauk,GRFx,GRFy = sym.symbols(['\\tau_{hip}','\\tau_{knee}','G_x','G_y']) 

# STEP 1: position vectors ri = [x,y,theta] (world frame)
rb = sym.Matrix([[x],
                [y],
                [thb]])

rl1 = sym.Matrix([[x + 0.5*ll1*sym.sin(thb + thlt)],
                [y - 0.5*ll1*sym.cos(thb + thlt)],
                [thb + thlt]])

rl2 = sym.Matrix([[x + ll1*sym.sin(thb + thlt)+0.5*ll2*sym.sin(thb + thlt + thlb)],
                [y - ll1*sym.cos(thb + thlt) - 0.5*ll2*sym.cos(thb + thlt + thlb)],
                [thb + thlt + thlb]])

# the Jacobians
Jb = rb.jacobian(q)
Jl1 = rl1.jacobian(q)
Jl2 = rl2.jacobian(q)

# STEP 2: generate expressions for the system space velocities from the jacobians
vb = Jb*dq
vl1 = Jl1*dq
vl2 = Jl2*dq

# STEP 3: generate expressions for the kinetic and potential energy
# mass vectors
Mb = sym.Matrix([[mb,mb,Inb]])
Ml1 = sym.Matrix([[ml1,ml1,Inl1]])
Ml2 = sym.Matrix([[ml2,ml2,Inl2]])

T = 0.5*Mb*sym.matrix_multiply_elementwise(vb,vb) + 0.5*Ml1*sym.matrix_multiply_elementwise(vl1,vl1) + 0.5*Ml2*sym.matrix_multiply_elementwise(vl2,vl2)
T = T[0]
V = mb*g*rb[1] + ml1*g*rl1[1] + ml2*g*rl2[1]


# STEP 4: calculate each term of the Lagrange equation
# term 1
Lg1 = sym.zeros(len(q),1)
for i in range(len(q)):
    dT_ddq = sym.Matrix([sym.diff(T,dq[i])]) # get partial of T in dq_i
    Lg1[i] = dT_ddq.jacobian(q)*dq + dT_ddq.jacobian(dq)*ddq #...then get time derivative of that partial

# term 3
Lg3 = sym.Matrix([T]).jacobian(q).transpose() # partial of T in q

# term 4
Lg4 = sym.Matrix([V]).jacobian(q).transpose() # partial of U in q

# STEP 5: generalized forces

# force vectors for each link
tau_b = sym.Matrix([[0],[0],[-tauh]])
tau_l1 = sym.Matrix([[0],[0],[tauh-tauk]])
tau_l2 = sym.Matrix([[0],[0],[tauk]])

GRF_l2 = sym.Matrix([[GRFx],[GRFy],[0.5*ll2*GRFx*sym.cos(thb+thlt+thlb)+0.5*ll2*GRFy*sym.sin(thb+thlt+thlb)]])

Q = sym.zeros(len(q),1)
for j in range(len(q)):
    Q[j] = tau_b.transpose()*Jb[:,j]+(tau_l1).transpose()*Jl1[:,j]+(tau_l2+GRF_l2).transpose()*Jl2[:,j]

# AND combine!
EOM = Lg1 - Lg3 + Lg4 - Q

EOMs = sym.zeros(len(q),1)
for j in range(len(q)):
    EOMs[j] = EOM[j].simplify()

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [3]:
# sym.solve(EOMs[4], ddq[4])

In [4]:
# Lambdify
from pyomo.environ import*
from pyomo.opt import SolverFactory
from pyomo.opt import SolverStatus, TerminationCondition
import time

func_map = {'sin':sin, 'cos':cos} 

sym_list = [g,mb,ml1,ml2,lb,ll1,ll2,Inb,Inl1,Inl2,
            x,y,thb,thlt,thlb,
            dx,dy,dthb,dthlt,dthlb,
            ddx,ddy,ddthb,ddthlt,ddthlb,
            tauh,tauk,GRFx,GRFy]
            
lambEOM_x = sym.lambdify(sym_list,EOMs[0],modules = [func_map])
lambEOM_y = sym.lambdify(sym_list,EOMs[1],modules = [func_map])
lambEOM_thb = sym.lambdify(sym_list,EOMs[2],modules = [func_map])
lambEOM_thlt = sym.lambdify(sym_list,EOMs[3],modules = [func_map])
lambEOM_thlb = sym.lambdify(sym_list,EOMs[4],modules = [func_map])

In [5]:
# rerun from here if you don't want to calculate the EOM's again
if 'm' in globals():
    del m # deletes the model
    
m = ConcreteModel()

# SETS-----------------------------------------------------------------------------------------------------------------------

T = 0.5 # total time
hm = 0.01 # master time step
N = int(T/hm) # number of nodes
m.N = RangeSet(N) 

links = [('body',1),('leg',1),('leg',2)]
m.L = Set(dimen=2, initialize = links)

DOFs = ['x','y','theta_b','theta_lt','theta_lb'] # generalized coordinates
m.DOF = Set(initialize = DOFs) 

# PARAMETERS-----------------------------------------------------------------------------------------------------------------

m.g = Param(initialize = 9.81)

def get_m(n, lb, ln):
    if lb == 'body':
        return 5.0
    else: return 2.5
m.m = Param(m.L, initialize = get_m) # mass of links

def get_len(n, lb, ln):
    if lb == 'body':
        return 1.0
    else: return 0.5
m.len = Param(m.L, initialize = get_len) # length of links

def calculate_In(m, lb, ln): 
    l = (lb,ln)
    # yes, that does mean you have to rebuild the tuple inside the function. Yes, it is dumb.
    return m.m[l]*m.len[l]**2/12 
m.In = Param(m.L, initialize = calculate_In) # moment of inertia

mbody = sum(m.m[l] for l in links)
BW = mbody*m.g.value

# VARIABLES -----------------------------------------------------------------------------------------------------------------

# system coordinates
m.q = Var(m.N, m.DOF) # position
m.dq = Var(m.N, m.DOF) # velocity
m.ddq = Var(m.N, m.DOF) # acceleration

# bound variables
for n in range(1,N+1):
    m.q[n,'y'].setlb(0.0)

In [6]:
# TIME AND INTEGRATION

# variable timestep
# hm = 0.02 # master timestep
m.h = Var(m.N, bounds = (0.8,1.2))

# Integration constraints 
def BwEuler_p(m,n,dof): # for positions
    if n > 1:
        return m.q[n,dof] == m.q[n-1,dof] + hm*m.h[n]*m.dq[n,dof]
    else:
        return Constraint.Skip
m.integrate_p = Constraint(m.N, m.DOF, rule = BwEuler_p)

def BwEuler_v(m,n,dof): # for velocities
    if n > 1:
        return m.dq[n,dof] == m.dq[n-1,dof] + hm*m.h[n]*m.ddq[n-1,dof]
    else:
        return Constraint.Skip 
m.integrate_v = Constraint(m.N, m.DOF, rule = BwEuler_v)

In [7]:
# GROUND INTERACTIONS -------------------------------------------------------------------------------------------------------

# paramters
m.mu = Param(initialize = 1.0) # friction coefficient

# sign set for positive and negative components
signs = ['ps','ng'] 
m.sgn = Set(initialize = signs)

WDOFs = ['X','Y',"THETA"] # absolute coordinates (see what I mean about switching between frames the whole time...?)
m.WDOF = Set(initialize = WDOFs) 

# variables
m.footp = Var(m.N, m.WDOF, bounds = (0.0,None)) # foot position
m.footv = Var(m.N, m.WDOF, m.sgn, bounds = (0.0,None)) # foot velocity

m.friction_cone = Var(m.N, bounds = (0.0,None))

m.GRF = Var(m.N, m.WDOF, m.sgn, bounds = (0.0,None)) # ground reaction forces

ground_constraints = ['contact','friction','slip_ps','slip_ng'] 
# ground_constraints = ['contact'] 
m.ground_constraints = Set(initialize = ground_constraints) # set for indexing ground-related penalties
m.ground_penalty = Var(m.N, m.ground_constraints, bounds = (0.0,None))

# constraints: aux variables
def def_footp(m,n,dof):
    if dof == 'Y':
        return m.footp[n,dof] == m.q[n,'y'] - m.len[('leg',1)]*cos(m.q[n,'theta_b']+m.q[n,'theta_lt']) - m.len[('leg',2)]*cos(m.q[n,'theta_b']+m.q[n,'theta_lt']+m.q[n,'theta_lb'])
    else:
        return Constraint.Skip
m.def_footp = Constraint(m.N, m.WDOF, rule = def_footp)

# lambdify the foot velocity
footx = sym.Matrix([x + ll1*sym.sin(thb + thlt) + ll2*sym.sin(thb + thlt + thlb)])
footdx = footx.jacobian(q)*dq
footdx = footdx[0].simplify()
lamb_footdx = sym.lambdify(sym_list,footdx,modules = [func_map])

def def_footv(m,n,dof):
    if dof == 'X':
        var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
            m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
            m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
            m.q[n,'x'],m.q[n,'y'],m.q[n,'theta_b'],m.q[n,'theta_lt'],m.q[n,'theta_lb'],
            m.dq[n,'x'],m.dq[n,'y'],m.dq[n,'theta_b'],m.dq[n,'theta_lt'],m.dq[n,'theta_lb'],
            m.ddq[n,'x'],m.ddq[n,'y'],m.ddq[n,'theta_b'],m.ddq[n,'theta_lt'],m.ddq[n,'theta_lb'],
            0,0,0,0]
        return m.footv[n,dof,'ps']-m.footv[n,dof,'ng'] == lamb_footdx(*var_list)
    else:
        return Constraint.Skip
m.def_footv = Constraint(m.N, m.WDOF, rule = def_footv)

def def_friction_cone(m,n):
    return m.friction_cone[n] == m.mu*m.GRF[n,'Y','ps'] - (m.GRF[n,'X','ps'] + m.GRF[n,'X','ng'])
m.def_friction_cone = Constraint(m.N, rule = def_friction_cone)

# constraints: complementarity

# contact
def ground_contact(m,n):
    if n < N:
        return m.ground_penalty[n,'contact'] == m.footp[n+1,'Y']*m.GRF[n,'Y','ps'] 
    else:
        return Constraint.Skip
m.ground_contact = Constraint(m.N, rule = ground_contact)

# friction
def ground_friction(m,n):
    return m.ground_penalty[n,'friction'] == (m.footv[n,'X','ps']+m.footv[n,'X','ng'])*m.friction_cone[n]
m.ground_friction = Constraint(m.N, rule = ground_friction)

# slipping
def ground_slip_ps(m,n):
    return m.ground_penalty[n,'slip_ps'] == m.footv[n,'X','ps']*m.GRF[n,'X','ps']
m.ground_slip_ps = Constraint(m.N, rule = ground_slip_ps)

def ground_slip_ng(m,n):
    return m.ground_penalty[n,'slip_ng'] == m.footv[n,'X','ng']*m.GRF[n,'X','ng']
m.ground_slip_ng = Constraint(m.N, rule = ground_slip_ng)

# bound contact forces at last node
for dof in WDOFs:
    for sgn in signs:
        m.GRF[N,dof,sgn].value = 0
        m.GRF[N,dof,sgn].fixed = True

In [8]:
# HARD JOINT STOPS ----------------------------------------------------------------------------------------------------------

# sets
joints = ['hip','knee']
m.J = Set(initialize = joints)

joint_constraints = ['up','lo'] # set of joint penalties
m.joint_constraints = Set(initialize = joint_constraints)

# parameters
hip_bound = [-np.pi/2, np.pi/2]
m.hip_bound = Param(m.joint_constraints, initialize = {'up':hip_bound[1],'lo':hip_bound[0]}) 

knee_bound = [-np.pi, 0.0]
m.knee_bound = Param(m.joint_constraints, initialize = {'up':knee_bound[1],'lo':knee_bound[0]})

# we can bound the joint coordinates directly
for n in range(1,N+1):
    m.q[n,'theta_b'].setlb(-np.pi/2)
    m.q[n,'theta_b'].setub(np.pi/2)
    m.q[n,'theta_lt'].setlb(hip_bound[0])
    m.q[n,'theta_lt'].setub(hip_bound[1])
    m.q[n,'theta_lb'].setlb(knee_bound[0])
    m.q[n,'theta_lb'].setub(knee_bound[1])

# variables
m.tau_ah = Var(m.N, bounds = (-2,2)) # actuator torque at hip
m.tau_ak = Var(m.N, bounds = (-2,2)) # actuator torque at knee
m.tau_rh = Var(m.N, m.sgn, bounds = (0.0,None)) # rebound torque hip
m.tau_rk = Var(m.N, m.sgn, bounds = (0.0,None)) # rebound torque knee

m.joint_penalty = Var(m.N, m.J, m.joint_constraints, bounds = (0.0,None))

# complementarity
def hip_limits(m,n,jc):
    if n < N:
        if jc == 'up':
            # NEXT angle
            return m.joint_penalty[n,'hip',jc] == (m.hip_bound['up'] - m.q[n+1,'theta_lt'])*m.tau_rh[n,'ng']
        else:
            return m.joint_penalty[n,'hip',jc] == (m.q[n+1,'theta_lt'] - m.hip_bound['lo'])*m.tau_rh[n,'ps']
    else:
        return Constraint.Skip
m.hip_limits = Constraint(m.N, m.joint_constraints, rule = hip_limits)

def knee_limits(m,n,jc):
    if n < N:
        if jc == 'up':
            # NEXT angle
            return m.joint_penalty[n,'knee',jc] == (m.knee_bound['up'] - m.q[n+1,'theta_lb'])*m.tau_rk[n,'ng']
        else:
            return m.joint_penalty[n,'knee',jc] == (m.q[n+1,'theta_lb'] - m.knee_bound['lo'])*m.tau_rk[n,'ps']
    else:
        return Constraint.Skip
m.knee_limits = Constraint(m.N, m.joint_constraints, rule = knee_limits)

# bound contact forces at last node
for sgn in signs:
    m.tau_rh[N,sgn].fix(0.0)
    m.tau_rk[N,sgn].fix(0.0)

In [9]:
# COST FUNCTION -------------------------------------------------------------------------------------------------------------

# minimum time

def CostFun(m):
    T = sum(m.h[n] for n in range(1,N+1))
    penalty_sum = 0
    torque_sum = 0
    for n in range(1,N+1):
        for gc in ground_constraints:
            penalty_sum += m.ground_penalty[n,gc]
        for jc in joint_constraints:
            for j in joints:
                penalty_sum += m.joint_penalty[n,j,jc]
        torque_sum += (m.tau_ah[n]**2 + m.tau_ak[n]**2)*m.h[n] 
        
#     return T+1000*penalty_sum
    return torque_sum+1000*penalty_sum
m.Cost = Objective(rule = CostFun)

In [10]:
# EQUATIONS OF MOTION -------------------------------------------------------------------------------------------------------
S = BW

def EOM_x(m,n):
    tau_inh = S*(m.tau_ah[n])
    tau_ink = S*(m.tau_ak[n])
    Gx_in = S*(m.GRF[n,'X','ps']-m.GRF[n,'X','ng'])
    Gy_in = S*(m.GRF[n,'Y','ps'])
    
    var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
            m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
            m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
            m.q[n,'x'],m.q[n,'y'],m.q[n,'theta_b'],m.q[n,'theta_lt'],m.q[n,'theta_lb'],
            m.dq[n,'x'],m.dq[n,'y'],m.dq[n,'theta_b'],m.dq[n,'theta_lt'],m.dq[n,'theta_lb'],
            m.ddq[n,'x'],m.ddq[n,'y'],m.ddq[n,'theta_b'],m.ddq[n,'theta_lt'],m.ddq[n,'theta_lb'],
            tau_inh,tau_ink,Gx_in,Gy_in]
    return lambEOM_x(*var_list) == 0
m.EOM_x = Constraint(m.N, rule = EOM_x)

def EOM_y(m,n):
    tau_inh = S*(m.tau_ah[n])
    tau_ink = S*(m.tau_ak[n])
    Gx_in = S*(m.GRF[n,'X','ps']-m.GRF[n,'X','ng'])
    Gy_in = S*(m.GRF[n,'Y','ps'])
    
    var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
            m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
            m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
            m.q[n,'x'],m.q[n,'y'],m.q[n,'theta_b'],m.q[n,'theta_lt'],m.q[n,'theta_lb'],
            m.dq[n,'x'],m.dq[n,'y'],m.dq[n,'theta_b'],m.dq[n,'theta_lt'],m.dq[n,'theta_lb'],
            m.ddq[n,'x'],m.ddq[n,'y'],m.ddq[n,'theta_b'],m.ddq[n,'theta_lt'],m.ddq[n,'theta_lb'],
            tau_inh,tau_ink,Gx_in,Gy_in]
    return lambEOM_y(*var_list) == 0
m.EOM_y = Constraint(m.N, rule = EOM_y)

def EOM_thb(m,n):
    tau_inh = S*(m.tau_ah[n])
    tau_ink = S*(m.tau_ak[n])
    Gx_in = S*(m.GRF[n,'X','ps']-m.GRF[n,'X','ng'])
    Gy_in = S*(m.GRF[n,'Y','ps'])
    
    var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
            m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
            m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
            m.q[n,'x'],m.q[n,'y'],m.q[n,'theta_b'],m.q[n,'theta_lt'],m.q[n,'theta_lb'],
            m.dq[n,'x'],m.dq[n,'y'],m.dq[n,'theta_b'],m.dq[n,'theta_lt'],m.dq[n,'theta_lb'],
            m.ddq[n,'x'],m.ddq[n,'y'],m.ddq[n,'theta_b'],m.ddq[n,'theta_lt'],m.ddq[n,'theta_lb'],
            tau_inh,tau_ink,Gx_in,Gy_in]
    return lambEOM_thb(*var_list) == 0
m.EOM_thb = Constraint(m.N, rule = EOM_thb)

def EOM_thlt(m,n):
    tau_inh = S*(m.tau_ah[n])
    tau_ink = S*(m.tau_ak[n])
    Gx_in = S*(m.GRF[n,'X','ps']-m.GRF[n,'X','ng'])
    Gy_in = S*(m.GRF[n,'Y','ps'])
    
    var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
            m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
            m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
            m.q[n,'x'],m.q[n,'y'],m.q[n,'theta_b'],m.q[n,'theta_lt'],m.q[n,'theta_lb'],
            m.dq[n,'x'],m.dq[n,'y'],m.dq[n,'theta_b'],m.dq[n,'theta_lt'],m.dq[n,'theta_lb'],
            m.ddq[n,'x'],m.ddq[n,'y'],m.ddq[n,'theta_b'],m.ddq[n,'theta_lt'],m.ddq[n,'theta_lb'],
            tau_inh,tau_ink,Gx_in,Gy_in]
    return lambEOM_thlt(*var_list) == 0
m.EOM_thlt = Constraint(m.N, rule = EOM_thlt)

def EOM_thlb(m,n):
    tau_inh = S*(m.tau_ah[n])
    tau_ink = S*(m.tau_ak[n])
    Gx_in = S*(m.GRF[n,'X','ps']-m.GRF[n,'X','ng'])
    Gy_in = S*(m.GRF[n,'Y','ps'])
    
    var_list = [m.g,m.m[('body',1)],m.m[('leg',1)],m.m[('leg',2)],
            m.len[('body',1)],m.len[('leg',1)],m.len[('leg',2)],
            m.In[('body',1)],m.In[('leg',1)],m.In[('leg',2)],
            m.q[n,'x'],m.q[n,'y'],m.q[n,'theta_b'],m.q[n,'theta_lt'],m.q[n,'theta_lb'],
            m.dq[n,'x'],m.dq[n,'y'],m.dq[n,'theta_b'],m.dq[n,'theta_lt'],m.dq[n,'theta_lb'],
            m.ddq[n,'x'],m.ddq[n,'y'],m.ddq[n,'theta_b'],m.ddq[n,'theta_lt'],m.ddq[n,'theta_lb'],
            tau_inh,tau_ink,Gx_in,Gy_in]
    return lambEOM_thlb(*var_list) == 0
m.EOM_thlb = Constraint(m.N, rule = EOM_thlb)

In [11]:
def def_q_periodic(m, dof): # periodicity in position
    if dof == 'x':
        return Constraint.Skip
    else:
        return m.q[1, dof] == m.q[N, dof]
m.def_q_periodic = Constraint(m.DOF, rule = def_q_periodic)

def def_dq_periodic(m, dof): # periodicity in velocity
    return m.dq[1, dof] == m.dq[N, dof]
m.def_dq_periodic = Constraint(m.DOF, rule = def_dq_periodic)

In [35]:
# Define periodic gate stuff
import random

x_0_lb = -1.0 # -1 m
x_0_ub = 1.0 # 1 m
penalty_thresh = 1e-6
iterations = 1000

# Periodic Gate -------------------------------------------------------------------------------------------------------------

opt = SolverFactory('ipopt',executable = '/home/alex/CoinIpoptBackup/build/bin/ipopt')
opt.options["linear_solver"] = 'ma86'

# solver options
opt.options["expect_infeasible_problem"] = 'yes'
#opt.options["linear_system_scaling"] = 'none'
#opt.options["mu_strategy"] = "adaptive"
opt.options["print_level"] = 5 # prints a log with each iteration (you want to this - it's the only way to see progress.)
opt.options["max_iter"] = 30000 # maximum number of iterations
opt.options["max_cpu_time"] = 120 # maximum cpu time in seconds
opt.options["Tol"] = 1e-6 # the tolerance for feasibility. Considers constraints satisfied when they're within this margin.


In [36]:
print('{0:<20s}{1:<20s}{2:<25s}{3:<20s}'.format("Iteration","Solved","Penalty","Time (s)"))
failed_solves = 0
success_solves = 0

for i in range(1,iterations+1):
    if 'm_sim' in globals():
        del m_sim
    
    m_sim = m.clone() # create a clean model
    
    m_sim.q[1,'x'].fix(np.random.uniform(x_0_lb, x_0_ub)) # initial condition
    
    vx_avg = float(random.randint(-4.0,4.0)) # 4 m/s
 
    def def_vx (m_sim):
        return vx_avg == (m_sim.q[N,'x'] - m_sim.q[1,'x'])/sum(m_sim.h[:])/hm
    m_sim.def_vx = Constraint(rule = def_vx)
    
    try:
        start = time.time()
        # solving
        results = opt.solve(m_sim, tee = False) 
        end = time.time()
    
    except:
        print("Something went wrong with solve...")
        failed_solves += 1
        continue
    
    if str(results.solver.termination_condition) == "optimal":
        penalty_sum = 0
        for n in range(1,N+1):
            for gc in ground_constraints:
                penalty_sum += m_sim.ground_penalty[n,gc].value
            for jc in joint_constraints:
                for j in joints:
                    penalty_sum += m_sim.joint_penalty[n,j,jc].value
    
    else: 
        penalty_sum = 0
    
    print('{0:<20s}{1:<20s}{2:<25}{3:<20d}'.format(str(i)+"/"+str(iterations), str(results.solver.termination_condition),penalty_sum,round(end-start)))
    
    if str(results.solver.termination_condition) == "optimal" and penalty_sum < penalty_thresh:
        qx_ar = np.array([m_sim.q[i,'x'].value for i in range(1, N+1)])
        qy_ar = np.array([m_sim.q[i,'y'].value for i in range(1, N+1)])
        qthb_ar = np.array([m_sim.q[i,'theta_b'].value for i in range(1, N+1)])
        qthlt_ar = np.array([m_sim.q[i,'theta_lt'].value for i in range(1, N+1)])
        qthlb_ar = np.array([m_sim.q[i,'theta_lb'].value for i in range(1, N+1)])
        tauh_ar = np.array([m_sim.tau_ah[i].value for i in range(1, N+1)])
        tauk_ar = np.array([m_sim.tau_ak[i].value for i in range(1, N+1)])

        if vx_avg >= 0:
            vx_avg_str = "p" + str(int(vx_avg))
        elif vx_avg < 0:
            vx_avg_str = "n" + str(abs(int(vx_avg)))

        filename = "iter_" + str(i) + "_" + vx_avg_str + "ms.npy"

        filepath = "./Datasets/test1/"

        file = filepath + filename
        np.save(file, [qx_ar, qy_ar, qthb_ar, qthlt_ar, qthlb_ar, vx_avg, tauh_ar, tauk_ar])
        print("Saved file:", file)
        success_solves += 1
    else:
        print("Failed")
        failed_solves += 1
#     m.del_component(m_sim.def_vx)


print("---------------------------------------------------------------------")
print("Batch finished.")
print("Percetage sucessful solves:", round(success_solves/iterations*100), "%")


Iteration           Solved              Penalty                  Time (s)            
1/1000              optimal             4.247688143844644e-08    5                   
Saved file: ./Datasets/test1/iter_1_p0ms.npy
2/1000              optimal             3.0620791504454485e-08   48                  
Saved file: ./Datasets/test1/iter_2_n1ms.npy
Something went wrong with solve...
4/1000              optimal             4.443191930882187e-08    4                   
Saved file: ./Datasets/test1/iter_4_p0ms.npy
5/1000              optimal             2.2351394048755643e-08   42                  
Saved file: ./Datasets/test1/iter_5_p2ms.npy
Something went wrong with solve...
7/1000              optimal             2.4262743139890235e-08   29                  
Saved file: ./Datasets/test1/iter_7_n4ms.npy
8/1000              optimal             2.6218502488332296e-08   31                  
Saved file: ./Datasets/test1/iter_8_n4ms.npy
9/1000              optimal             2.8647035191122876

70/1000             optimal             1.921481191594238e-08    25                  
Saved file: ./Datasets/test1/iter_70_n4ms.npy
71/1000             optimal             1.6895536198379582e-07   41                  
Saved file: ./Datasets/test1/iter_71_n1ms.npy
72/1000             optimal             1.2414047507837809e-08   31                  
Saved file: ./Datasets/test1/iter_72_n3ms.npy
Something went wrong with solve...
74/1000             optimal             3.357232673186063e-08    6                   
Saved file: ./Datasets/test1/iter_74_p0ms.npy
75/1000             optimal             4.2663407966881105e-08   25                  
Saved file: ./Datasets/test1/iter_75_p1ms.npy
76/1000             optimal             1.4355597439688424e-08   31                  
Saved file: ./Datasets/test1/iter_76_n2ms.npy
77/1000             optimal             9.13217526360448e-09     24                  
Saved file: ./Datasets/test1/iter_77_p2ms.npy
78/1000             optimal             3

137/1000            optimal             1.3947228496570019e-08   30                  
Saved file: ./Datasets/test1/iter_137_p4ms.npy
138/1000            optimal             4.80469123431227e-08     1                   
Saved file: ./Datasets/test1/iter_138_p0ms.npy
139/1000            optimal             2.330954050860129e-08    43                  
Saved file: ./Datasets/test1/iter_139_n2ms.npy
140/1000            optimal             8.660191803861106e-09    26                  
Saved file: ./Datasets/test1/iter_140_p2ms.npy
141/1000            optimal             1.1175035017904187e-08   25                  
Saved file: ./Datasets/test1/iter_141_n3ms.npy
142/1000            optimal             3.605603781756365e-08    29                  
Saved file: ./Datasets/test1/iter_142_n1ms.npy
143/1000            optimal             1.6625361376672346e-08   27                  
Saved file: ./Datasets/test1/iter_143_p2ms.npy
144/1000            optimal             1.2954125841359115e-08   28  

Something went wrong with solve...
202/1000            optimal             2.7033844710545164e-08   35                  
Saved file: ./Datasets/test1/iter_202_n2ms.npy
203/1000            optimal             3.672786182219124e-08    35                  
Saved file: ./Datasets/test1/iter_203_n3ms.npy
204/1000            optimal             3.75588843990674e-08     44                  
Saved file: ./Datasets/test1/iter_204_n1ms.npy
205/1000            optimal             1.8943323372857468e-08   43                  
Saved file: ./Datasets/test1/iter_205_p4ms.npy
206/1000            optimal             5.612802632654048e-08    27                  
Saved file: ./Datasets/test1/iter_206_p1ms.npy
207/1000            optimal             3.626154999503469e-08    40                  
Saved file: ./Datasets/test1/iter_207_p1ms.npy
208/1000            optimal             4.631739083221702e-08    3                   
Saved file: ./Datasets/test1/iter_208_p0ms.npy
209/1000            optimal       

264/1000            optimal             1.442161301147999e-08    26                  
Saved file: ./Datasets/test1/iter_264_n3ms.npy
265/1000            optimal             2.0215180135177112e-08   19                  
Saved file: ./Datasets/test1/iter_265_n4ms.npy
266/1000            optimal             3.1117675436450223e-08   24                  
Saved file: ./Datasets/test1/iter_266_p1ms.npy
267/1000            optimal             9.033729310268774e-09    35                  
Saved file: ./Datasets/test1/iter_267_p4ms.npy
268/1000            optimal             4.834530991937832e-08    1                   
Saved file: ./Datasets/test1/iter_268_p0ms.npy
269/1000            optimal             4.2451442436191577e-08   5                   
Saved file: ./Datasets/test1/iter_269_p0ms.npy
270/1000            optimal             4.7510000094596514e-08   32                  
Saved file: ./Datasets/test1/iter_270_p1ms.npy
271/1000            optimal             3.807572044136317e-08    51  

...

723/1000            optimal             9.431533057799013e-09    30                  
Saved file: ./Datasets/test1/iter_723_n4ms.npy
724/1000            optimal             1.3969019477634957e-08   27                  
Saved file: ./Datasets/test1/iter_724_p3ms.npy
725/1000            optimal             9.802744744350607e-09    35                  
Saved file: ./Datasets/test1/iter_725_p4ms.npy
726/1000            optimal             1.3725082676979552e-08   36                  
Saved file: ./Datasets/test1/iter_726_p3ms.npy
727/1000            optimal             4.083510695792737e-08    37                  
Saved file: ./Datasets/test1/iter_727_n1ms.npy
728/1000            optimal             4.542191486530905e-08    4                   
Saved file: ./Datasets/test1/iter_728_p0ms.npy
729/1000            optimal             3.746729468647127e-08    7                   
Saved file: ./Datasets/test1/iter_729_p0ms.npy
730/1000            optimal             2.8429514172896942e-08   34  

787/1000            optimal             2.8661679311543472e-08   37                  
Saved file: ./Datasets/test1/iter_787_p1ms.npy
Something went wrong with solve...
789/1000            optimal             4.885733316833597e-08    36                  
Saved file: ./Datasets/test1/iter_789_n3ms.npy
790/1000            optimal             3.3226247639548275e-08   42                  
Saved file: ./Datasets/test1/iter_790_n3ms.npy
791/1000            optimal             1.204541669277592e-08    32                  
Saved file: ./Datasets/test1/iter_791_p3ms.npy
792/1000            optimal             2.741620048578376e-08    44                  
Saved file: ./Datasets/test1/iter_792_n2ms.npy
793/1000            optimal             3.357232673186778e-08    9                   
Saved file: ./Datasets/test1/iter_793_p0ms.npy
794/1000            optimal             9.888023565089161e-09    32                  
Saved file: ./Datasets/test1/iter_794_p4ms.npy
795/1000            optimal       

851/1000            optimal             1.974328484923556e-08    48                  
Saved file: ./Datasets/test1/iter_851_n1ms.npy
852/1000            optimal             3.4559057485492015e-08   4                   
Saved file: ./Datasets/test1/iter_852_p0ms.npy
853/1000            optimal             1.08071517857658e-08     34                  
Saved file: ./Datasets/test1/iter_853_p4ms.npy
854/1000            optimal             3.475657749805779e-08    43                  
Saved file: ./Datasets/test1/iter_854_n1ms.npy
855/1000            optimal             3.287591022819178e-08    48                  
Saved file: ./Datasets/test1/iter_855_n2ms.npy
856/1000            optimal             2.3702674475387824e-08   30                  
Saved file: ./Datasets/test1/iter_856_p1ms.npy
857/1000            optimal             8.521918370329032e-09    32                  
Saved file: ./Datasets/test1/iter_857_n4ms.npy
858/1000            optimal             8.994122976219079e-09    26  

918/1000            optimal             1.0087240387761148e-08   26                  
Saved file: ./Datasets/test1/iter_918_n3ms.npy
919/1000            optimal             2.8051047758495488e-08   34                  
Saved file: ./Datasets/test1/iter_919_n3ms.npy
920/1000            optimal             1.548206169550464e-08    35                  
Saved file: ./Datasets/test1/iter_920_n2ms.npy
921/1000            optimal             1.0319887428085442e-08   23                  
Saved file: ./Datasets/test1/iter_921_p3ms.npy
922/1000            optimal             3.6641924813578534e-08   31                  
Saved file: ./Datasets/test1/iter_922_p1ms.npy
923/1000            optimal             4.2722068690671365e-08   41                  
Saved file: ./Datasets/test1/iter_923_n2ms.npy
924/1000            optimal             4.3721318049703047e-08   56                  
Saved file: ./Datasets/test1/iter_924_n2ms.npy
925/1000            optimal             4.281560664362003e-08    28  

982/1000            optimal             1.1091121535885638e-08   26                  
Saved file: ./Datasets/test1/iter_982_p3ms.npy
983/1000            optimal             7.316182991428366e-09    24                  
Saved file: ./Datasets/test1/iter_983_n4ms.npy
984/1000            optimal             3.455878457474489e-08    5                   
Saved file: ./Datasets/test1/iter_984_p0ms.npy
985/1000            optimal             2.4414611651244857e-08   82                  
Saved file: ./Datasets/test1/iter_985_n2ms.npy
986/1000            optimal             3.858219849040129e-08    34                  
Saved file: ./Datasets/test1/iter_986_n3ms.npy
987/1000            optimal             3.754516162189829e-08    36                  
Saved file: ./Datasets/test1/iter_987_p1ms.npy
988/1000            optimal             2.2084822401291443e-08   46                  
Saved file: ./Datasets/test1/iter_988_n1ms.npy
Something went wrong with solve...
990/1000            optimal       

In [30]:
# m_sim = m.clone()

# m_sim.q.pprint()

# round(success_solves/iterations*100)

In [ ]:
# print(results.solver.status) # tells you if the solver had any errors/ warnings
# print(results.solver.termination_condition) # tells you if the solution was (locally) optimal, feasible, or neither.

# penalty_sum = 0
# for n in range(1,N+1):
#     for gc in ground_constraints:
#         penalty_sum += m.ground_penalty[n,gc].value
#     for jc in joint_constraints:
#         for j in joints:
#             penalty_sum += m.joint_penalty[n,j,jc].value


# print()
# print("Penalty:   ", penalty_sum)
# print("Solve Time: %.2f" %solve_time, "s", sep="")
# # m.pprint() 

In [34]:
#animate it
import matplotlib.pyplot as plt
import matplotlib.animation as ani
from IPython.display import HTML
%matplotlib inline

fig1, ax1 = plt.subplots(1,1) #create axes
ax1.set_aspect('equal')

def plot_robot(i,m_sim,ax): #update function for animation
    ax.clear()
    ax.set_xlim([-5.0,5.0])
    ax.set_ylim([0,4])
    
    #plot body
    body_xb = m_sim.q[i,'x'].value - 0.5*m_sim.len[('body',1)]*cos(m_sim.q[i,'theta_b'].value)
    body_yb = m_sim.q[i,'y'].value - 0.5*m_sim.len[('body',1)]*sin(m_sim.q[i,'theta_b'].value)
    body_xf = m_sim.q[i,'x'].value + 0.5*m_sim.len[('body',1)]*cos(m_sim.q[i,'theta_b'].value)
    body_yf = m_sim.q[i,'y'].value + 0.5*m_sim.len[('body',1)]*sin(m_sim.q[i,'theta_b'].value)  
    ax.plot([body_xb,body_xf],[body_yb,body_yf],color='xkcd:black')
      
    #plot leg 1
    thA1 = m_sim.q[i,'theta_b'].value+m_sim.q[i,'theta_lt'].value
    thA2 = thA1+m_sim.q[i,'theta_lb'].value
    leg1_xt = m_sim.q[i,'x'].value
    leg1_yt = m_sim.q[i,'y'].value
    leg1_xb = m_sim.q[i,'x'].value + m_sim.len[('leg',1)]*sin(thA1)
    leg1_yb = m_sim.q[i,'y'].value - m_sim.len[('leg',1)]*cos(thA1)
    ax.plot([leg1_xt,leg1_xb],[leg1_yt,leg1_yb],color='xkcd:black')
    
    #plot leg 2
    leg2_xt = leg1_xb
    leg2_yt = leg1_yb
    leg2_xb = leg1_xb + m_sim.len[('leg',2)]*sin(thA2)
    leg2_yb = leg1_yb - m_sim.len[('leg',2)]*cos(thA2)
    ax.plot([leg2_xt,leg2_xb],[leg2_yt,leg2_yb],color='xkcd:black')
    
update = lambda i: plot_robot(i,m_sim,ax1) #lambdify update function

animate = ani.FuncAnimation(fig1,update,range(1,N+1),interval = 50,repeat=True)
plt.close(animate._fig)

HTML(animate.to_html5_video()) #you need to convert the animation to HTML5 to embed it in the notebook


In [181]:
# m.q.pprint()

In [235]:
# qx_ar = np.array([m.q[i,'x'].value for i in range(1, N+1)])
# qy_ar = np.array([m.q[i,'y'].value for i in range(1, N+1)])
# qthb_ar = np.array([m.q[i,'theta_b'].value for i in range(1, N+1)])
# qthlt_ar = np.array([m.q[i,'theta_lt'].value for i in range(1, N+1)])
# qthlb_ar = np.array([m.q[i,'theta_lb'].value for i in range(1, N+1)])

# vx_avg = 0.0
# i = 1
# if vx_avg > 0:
#     vx_avg_str = "p" + str(int(vx_avg))
# elif vx_avg < 0:
#     vx_avg_str = "n" + str(abs(int(vx_avg)))
# else: 
#     vx_avg_str = str(int(vx_avg)) 
    
# filename = "iter_" + str(i) + "_" + vx_avg_str + "ms.npy"

# filepath = "./Datasets/test1/"

# file = filepath + filename

# np.save(file, [qx_ar, qy_ar, qthb_ar, qthlt_ar, qthlb_ar, vx_avg])

In [236]:
one, two, three, four, five, six = np.load(file, allow_pickle = True)

In [238]:
# six